In [1]:
from dotenv import load_dotenv
load_dotenv
import os

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-4.1-mini",
    temperature=0.2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
result = llm.invoke("What is an agent?")

In [4]:
print(result)

content='An **agent** is an entity that can perceive its environment through sensors and act upon that environment through actuators to achieve specific goals. In various contexts, the term "agent" can have slightly different meanings:\n\n1. **In Artificial Intelligence (AI):**  \n   An agent is a software program or system that autonomously performs tasks by perceiving its environment and taking actions to achieve objectives. For example, a robotic vacuum cleaner is an agent that senses dirt and obstacles and moves accordingly to clean a room.\n\n2. **In Multi-Agent Systems:**  \n   An agent is an autonomous unit that interacts with other agents to solve problems or perform tasks collaboratively or competitively.\n\n3. **In Economics or Business:**  \n   An agent is a person or entity authorized to act on behalf of another (the principal), such as a real estate agent or a sales agent.\n\n4. **In General Use:**  \n   An agent can be any actor that causes an effect or change, such as a 

In [6]:
from langchain.tools import tool


@tool
def write_email(
    to: str,
    subject: str,
    content: str,
) -> str:
    """write and send an email"""
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [7]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [8]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [11]:
model_with_tool = llm.bind_tools(
    [write_email],
    tool_choice="any",
    parallel_tool_calls=False,
)
output=model_with_tool.invoke("Draft a response to my boss (boss@company.ai) about tomorrow's meeting")

In [12]:
type(output)

langchain_core.messages.ai.AIMessage

In [13]:
output

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 67, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_d0c93c37b1', 'id': 'chatcmpl-Co25Qp37bva6wzCBmtWlfzfQBJxsQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b3036-c90f-7660-a7ce-43b207ff4305-0', tool_calls=[{'name': 'write_email', 'args': {'to': 'boss@company.ai', 'subject': "Re: Tomorrow's Meeting", 'content': "Dear Boss,\n\nThank you for the reminder about tomorrow's meeting. Please let me know if there are any specific topics or materials I should prepare in advance.\n\nLooking forward to it.\n\nBest regards,\n[Your Name]"}, 'id': 'c